In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE68608"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE68608"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE68608.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE68608.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE68608.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information about C9ORF72 ALS study with motor neurons
# This is likely a gene expression dataset looking at splicing dysregulation
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the sample characteristics dictionary
# For trait (ALS), row 1 contains "patient group" information
trait_row = 1

# There's no information about age in the sample characteristics
age_row = None

# No gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert ALS trait value to binary (1 for ALS, 0 for control)"""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary
    if 'ALS' in value or 'mutated C9ORF72' in value:
        return 1
    elif 'control' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value):
    """Convert age value to continuous"""
    # Not applicable as age data is not available
    return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not applicable as gender data is not available
    return None

# 3. Save Metadata
# Check trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract information from sample characteristics dictionary
    sample_ids = []
    for item in [s.split(": ")[1] for s in sample_chars[0]]:
        sample_ids.append(item)
    
    # Create a DataFrame with appropriate structure for geo_select_clinical_features
    data = []
    for sample_id in sample_ids:
        if 'Patient' in sample_id:
            # For patients
            data.append({
                'ID_REF': sample_id,
                trait_row: 'patient group: ALS due to mutated C9ORF72'
            })
        else:
            # For controls
            data.append({
                'ID_REF': sample_id,
                trait_row: 'patient group: Neurologically healthy, non-disease control'
            })
    
    # Create DataFrame
    clinical_df = pd.DataFrame(data)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement the code for the current step with corrections to address the file parsing issue:

```python
import pandas as pd
import numpy as np
import os
import json
import glob
import gzip
from typing import Optional, Callable, Dict, Any, List, Union

# Initialize variables for validation
is_gene_available = False
is_trait_available = False
trait_row = None
age_row = None
gender_row = None

try:
    # Look for any series matrix file (compressed or not)
    matrix_files = glob.glob(os.path.join(in_cohort_dir, "*series_matrix*.txt*"))
    
    if not matrix_files:
        print(f"No series matrix file found in {in_cohort_dir}")
        clinical_data = pd.DataFrame()  # Empty DataFrame if no file found
    else:
        matrix_file = matrix_files[0]  # Take the first matching file
        print(f"Found matrix file: {matrix_file}")
        
        # First, let's examine the file structure
        if matrix_file.endswith('.gz'):
            with gzip.open(matrix_file, 'rt') as f:
                header_lines = [next(f) for _ in range(40) if f]
        else:
            with open(matrix_file, 'rt') as f:
                header_lines = [next(f) for _ in range(40) if f]
        
        # Print a few header lines to understand the structure
        print("First few lines of the file:")
        for i, line in enumerate(header_lines[:5]):
            print(f"Line {i+1}: {line.strip()}")
        
        # Read the file with flexible parsing to handle potential formatting issues
        clinical_data = pd.read_csv(matrix_file, sep='\t', comment='#', nrows=70, 
                                    on_bad_lines='skip', engine='python')
        
        # Check if the file has content
        if clinical_data.empty:
            print("Warning: The matrix file is empty or couldn't be parsed properly")
        else:
            # Print the shape and first few columns to understand the structure
            print(f"Clinical data shape: {clinical_data.shape}")
            print("First column names:", clinical_data.columns[:5].tolist())
            
            # Examine the first column which typically contains metadata identifiers
            first_col = clinical_data.iloc[:,0].dropna().tolist()
            print("\nMetadata identifiers in first column:")
            for i, item in enumerate(first_col[:10]):  # Print first 10 items
                print(f"{i}: {item}")
            
            # Check if this contains gene expression data
            # Look for platform information and other indicators
            platform_entries = [item for item in first_col if 'platform' in str(item).lower()]
            if platform_entries:
                print("\nPlatform information:")
                for entry in platform_entries:
                    print(entry)
                # Typical gene expression platforms start with GPL
                if any('GPL' in str(entry) for entry in platform_entries):
                    is_gene_available = True
                    print("This appears to be gene expression data based on platform information")
            
            # Look for sample characteristics entries to identify clinical features
            sample_char_entries = [i for i, item in enumerate(first_col) 
                                  if 'sample_char' in str(item).lower()]
            
            if sample_char_entries:
                print("\nSample characteristic entries found at rows:", sample_char_entries)
                
                # Examine each sample characteristic row
                for idx in sample_char_entries:
                    row_content = str(clinical_data.iloc[idx, 0])
                    unique_values = set(clinical_data.iloc[idx, 1:].dropna())
                    print(f"Row {idx}: {row_content}")
                    print(f"Unique values: {unique_values}")
                    
                    # Identify trait, age, and gender information
                    row_content_lower = row_content.lower()
                    if ('disease' in row_content_lower or 'status' in row_content_lower or 
                        'diagnosis' in row_content_lower or 'als' in row_content_lower or
                        'amyotrophic' in row_content_lower or 'control' in row_content_lower):
                        if trait_row is None and len(unique_values) > 1:  # Ensure it's not a constant feature
                            trait_row = idx
                            print(f"Identified trait row at {idx}")
                    elif 'age' in row_content_lower:
                        if age_row is None and len(unique_values) > 1:
                            age_row = idx
                            print(f"Identified age row at {idx}")
                    elif 'gender' in row_content_lower or 'sex' in row_content_lower:
                        if gender_row is None and len(unique_values) > 1:
                            gender_row = idx
                            print(f"Identified gender row at {idx}")
            
            is_trait_available = trait_row is not None
            
            # Print final determinations
            print(f"\nFinal determinations:")
            print(f"Gene expression data available: {is_gene_available}")
            print(f"Trait data available: {is_trait_available}")
            print(f"Trait row: {trait_row}")
            print(f"Age row: {age_row}")
            print(f"Gender row: {gender_row}")

except Exception as e:
    print(f"Error processing file: {str(e)}")
    clinical_data = pd.DataFrame()  # Empty DataFrame on error

# Define conversion functions
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for Control, 1 for ALS)"""
    if value is None or pd.isna(value):
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'als' in value or 'amyotrophic lateral sclerosis' in value or 'patient' in value:
        return 1
    elif 'control' in value or 'normal' in value or 'healthy' in value:
        return 0
    return None

def convert_age(value: str) -> float:
    """Convert age value to float"""
    if value is None or pd.isna(value):
        return None
    value = str(value)
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        # Extract numeric part if it contains non-numeric characters
        import re
        numeric_part = re.search(r'\d+(\.\d+)?', value)
        if numeric_part:
            return float(numeric_part.group())
        return None
    except:
        return None

def convert_gender(value: str) -> int:
    """Convert gender value to binary (0 for Female, 1 for Male)"""
    if value is None or pd.isna(value):
        return None
    value = value.lower() if isinstance(value, str) else str(value).lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'female' in value or 'f' == value or value.strip() == 'f':
        return 0
    elif 'male' in value or 'm' == value or value.strip() == 'm':
        return 1
    return None

# Save Metadata
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# Clinical Feature Extraction
if trait_row is not None and not clinical_data.empty:
    try:
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll debug the code by addressing the syntax errors and completing the implementation properly.

```python
import os
import pandas as pd
import numpy as np
import json
from typing import Dict, Any, Optional, Callable
import glob

# Check if this dataset contains gene expression data
# Look for gene expression files in the directory
gene_expression_patterns = ['*_series_matrix.txt', '*gene*', '*expr*', '*.txt', '*.csv', '*.tsv', '*.gz']
gene_files = []
for pattern in gene_expression_patterns:
    gene_files.extend(glob.glob(os.path.join(in_cohort_dir, pattern)))

# Filter out files that might be clinical data
gene_files = [f for f in gene_files if 'clinical' not in f.lower() and 'phenotype' not in f.lower()]
is_gene_available = len(gene_files) > 0

# Try to identify clinical data files using different patterns
clinical_data = None
clinical_file_patterns = ['*clinical*', '*phenotype*', '*meta*', '*sample*', '*char*', '*series_matrix.txt']
clinical_files = []
for pattern in clinical_file_patterns:
    clinical_files.extend(glob.glob(os.path.join(in_cohort_dir, pattern)))

# Load the first available clinical data file
for file_path in clinical_files:
    try:
        if file_path.endswith('.txt'):
            # For series matrix files, we need to extract the sample characteristics
            with open(file_path, 'r') as f:
                lines = f.readlines()
                
            # Extract sample characteristic lines
            sample_chars = []
            for i, line in enumerate(lines):
                if line.startswith('!Sample_characteristics_ch1'):
                    sample_chars.append(line.strip())
            
            if sample_chars:
                # Process sample characteristics into a DataFrame
                char_data = {}
                for i, char in enumerate(sample_chars):
                    parts = char.split('\t')
                    if i == 0:
                        # Initialize columns with sample IDs
                        samples = [p.replace('"', '') for p in parts[1:]]
                        for sample in samples:
                            char_data[sample] = []
                    
                    # Add characteristics for each sample
                    values = [p.replace('"', '').replace('!Sample_characteristics_ch1: ', '') for p in parts[1:]]
                    
                    # Check if this is a new type of characteristic
                    if len(values) > 0:
                        characteristic_type = values[0].split(':')[0] if ':' in values[0] else f'characteristic_{i}'
                        if characteristic_type not in char_data:
                            char_data[characteristic_type] = []
                        
                        # Add this characteristic to each sample
                        for j, value in enumerate(values):
                            if j < len(samples):
                                char_data[samples[j]].append(value)
                
                # Convert to DataFrame
                clinical_data = pd.DataFrame(char_data)
                break
        else:
            # Try standard CSV loading for other file types
            clinical_data = pd.read_csv(file_path)
            break
    except Exception as e:
        print(f"Could not load {file_path}: {e}")
        continue

# Display what we found for debugging
if clinical_data is not None:
    print("Clinical data preview:")
    print(clinical_data.head())
    print("\nColumn names:", clinical_data.columns.tolist())
    
    # Check for trait, age, and gender information
    trait_row = None
    age_row = None
    gender_row = None
    
    # Analyze each column for clinical information
    for col in clinical_data.columns:
        values = clinical_data[col].astype(str).str.lower()
        unique_values = values.unique()
        
        # Look for trait information (ALS vs control)
        if (any(['als' in str(v) for v in unique_values]) or 
            any(['amyotrophic' in str(v) for v in unique_values])) and \
           (any(['control' in str(v) for v in unique_values]) or 
            any(['healthy' in str(v) for v in unique_values])):
            print(f"Found trait information in column: {col}")
            print(f"Unique values: {unique_values}")
            trait_row = clinical_data.columns.get_loc(col)
        
        # Look for age information
        if any(['age' in str(v) for v in unique_values]) or \
           any([str(v).replace('.', '', 1).isdigit() for v in unique_values if v != 'nan']):
            print(f"Found potential age information in column: {col}")
            print(f"Unique values: {unique_values}")
            age_row = clinical_data.columns.get_loc(col)
        
        # Look for gender information
        if any(['male' in str(v) for v in unique_values]) or \
           any(['female' in str(v) for v in unique_values]) or \
           any(['gender' in str(v) for v in unique_values]) or \
           any(['sex' in str(v) for v in unique_values]):
            print(f"Found gender information in column: {col}")
            print(f"Unique values: {unique_values}")
            gender_row = clinical_data.columns.get_loc(col)
else:
    print("No clinical data found in the directory.")
    trait_row = None
    age_row = None
    gender_row = None

# Define conversion functions based on observed data patterns
def convert_trait(value):
    if value is None:
        return None
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = str(value).lower() if value is not None else ""
    if "als" in value or "amyotrophic" in value or "disease" in value:
        return 1
    elif "control" in value or "healthy" in value or "normal" in value:
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to extract the numeric age
    if isinstance(value, str):
        # Extract digits from the string
        import re
        digits = re.findall(r'\d+\.?\d*', value)
        if digits:
            return float(digits[0])
    elif isinstance(value, (int, float)):
        return float(value)
    return None

def convert_gender(value):
    if value is None:
        return None
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    value = str(value).lower() if value is not None else ""
    if "female" in value or "f" == value:
        return 0
    elif "male" in value or "m" == value:
        return 1
    return None

# Save initial metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait data is available
if is_trait_available and clinical_data is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("Selected clinical data preview:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Save to file
    os.makedirs(os.path.dirname(out


### Step 5: Gene Data Extraction

In [ ]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True


### Step 6: Gene Identifier Review

In [ ]:
# These identifiers are in Affymetrix HG-U133 Plus 2.0 format (e.g., "1007_s_at"), 
# which are probe IDs rather than human gene symbols. 
# They need to be mapped to standard gene symbols.

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate columns for gene identifiers and gene symbols
# From the preview, we can see:
# - The 'ID' column contains probe identifiers (e.g., '1007_s_at') matching the gene expression data
# - The 'Gene Symbol' column contains the human gene symbols we need to map to

# 2. Get the gene mapping dataframe using the library function
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a preview of the mapping
print("\nGene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Print information about the result
print(f"\nAfter mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and aggregate duplicate genes
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nAfter normalization: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Save the gene expression data to a file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")